# Intro to Autograd

This notebook will get you started with autograd, a powerful tool for taking derivatives of arbitrary functions. The two most popular frameworks for implementing autograd are Pytorch and Tensorflow. We will be using Pytorch because it is more user-friendly.

This framework is particularly important for calculating derivatives of deep learning functions, but it can also be useful for classical models like logistic regression. This notebook covers a basic example of gradient descent in a 1D state space (you will do the same in 2D for your homework), and we will cover more complex uses of autograd (for example, to train a model) later on in the course.

In [1]:
# import the pytorch module (called torch)
import torch

# the basic unit of pytorch is a tensor (works almost the same as numpy array, but with extra features)
x_tensor = torch.tensor([-2.3, 4])
print(x_tensor)

# you can use scalars and built-in python math ops as you normally would
print(2 * (x_tensor + 1), x_tensor ** 3)
# for vector addition or multiplication, etc. must be tensor of same size
y_tensor = torch.tensor([3.4, -3])
print(x_tensor + y_tensor, x_tensor / y_tensor)

tensor([-2.3000,  4.0000])
tensor([-2.6000, 10.0000]) tensor([-12.1670,  64.0000])
tensor([1.1000, 1.0000]) tensor([-0.6765, -1.3333])


In [2]:
# a function that we want to minimize (must be written to accept and return a torch tensor)
def f(x_var):
    y = (x_var + torch.Tensor([1.0])) ** 2
    return y

# initialize x (we will start from x = 3.63)
x_0 = torch.tensor([3.63])

# gradient descent parameters
step_sz = 0.35
num_steps = 20

# make an autograd variable for x (after performing operations with x, the gradient can be passed back to x)
x = torch.autograd.Variable(x_0, requires_grad=True)
# if x.requires_grad is True, this means it is "activated" for autograd computation
print("Gradients can be calculated for operations that use x: ", x.requires_grad)

Gradients can be calculated for operations that use x:  True


In [3]:
# for loop for iterative gradient descent
for step in range(num_steps):
    # get the gradient (torch.autograd.grad returns 1-element tuple so need to grab element 0 of the output)
    x_grad = torch.autograd.grad(f(x), x)[0]
    
    # gradient descent update. (a -= b) translates to (a = a - b)
    x.data -= step_sz * x_grad

In [4]:
# remove x from the autograd process (not necessary for this example, but usually turn of autograd when done)
x = x.detach()
# if x.requires_grad is False, this means it is a normal tensor
print("Gradients can be calculated for operations that use x: ", x.requires_grad)
# print result. the minimizer of f is x = -1, which is exactly what we get (up to some small numerical error)
print("Final minimizer from gradient descent: x = ", x)

Gradients can be calculated for operations that use x:  False
Final minimizer from gradient descent: x =  tensor([-1.])


There are other ways to use autograd that we will look at later. The basic idea behind autograd is that a "computational graph" is built everytime that an operation is performed with a variable that has the `requires_grad=True` attribute. In this example, the graph starts at `x` and ends at `f(x)`. Once the end of the graph is reached, gradients can be passed from the end of the graph to beginning of the graph (or an intermediate point) using a technique called "backpropagation", whixh is a fancy term for a computataional implementaion of the Chain Rule from calculus. 

We will discuss the details more later, but for now feel free to play around with autograd or look into a more detailed explanation [here](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html). The Pytorch tutorial shows you how to do backpropagation using the `.backward()` function instead of `torch.autograd.grad`, but the idea is exactly the same.

## 2D arrays

Below is a demo for manipulating 2D variables in Python. The main ideas are very similar to working with arrays in other languages like R or C.

In [5]:
# let's define a function that takes a 2D variable and returns a scalar (like HW 1 Problem 3)
# specifically, we will implement g(x) = (x1 - 4)^2 + (x2 - 3)^2 for 2D x = (x1, x2)
# this is somewhat similar to your homework problem, except you will also need torch.exp
def g(x):
    # you can access elements of a vector using the square brackets
    # for Python, 0 corresponds to the 1st element, 1 corresponds to the 2nd element, etc.
    # Note that the index is different from R, which starts at 1 for the 1st element
    y = (x[0] - 4)**2 + (x[1] - 3)**2
    return y

# let's define a 2D variable 
x_var = torch.tensor([0., 0.])
# now let's evaluate g and print
print('x: ' , x_var, ' g(x): ', g(x_var))

# now let's get make an autograd variable and get the gradient
x_var = torch.autograd.Variable(x_var, requires_grad=True)
print(x_var)
grad = torch.autograd.grad(g(x_var), x_var)[0]
print('Gradient: ', grad)
# update with step size 0.1
x_var.data = x_var - 0.1 * grad
print('updated x: ', x_var)

x:  tensor([0., 0.])  g(x):  tensor(25.)
tensor([0., 0.], requires_grad=True)
Gradient:  tensor([-8., -6.])
updated x:  tensor([0.8000, 0.6000], requires_grad=True)


Below is an alternate but equivalent definition of g. The definition does element-wise operation before adding up the results. This one requires use of `torch.sum` to add up the elements of `x`. This time you don't need to take array sub-elements.

In [6]:
# specifically, we will implement g(x) = (x1 - 4)^2 + (x2 - 3)^2 for 2D x = (x1, x2)
# this is somewhat similar to your homework problem, except you will also need torch.exp
def g(x):
    # you can also do element-wise operations
    y = torch.sum((x - torch.tensor([4, 3]))**2)
    return y

# let's define a 2D variable 
x_var = torch.tensor([0., 0.])
# now let's evaluate g and print
print('x: ' , x_var, ' g(x): ', g(x_var))

# now let's get make an autograd variable and get the gradient
x_var = torch.autograd.Variable(x_var, requires_grad=True)
print(x_var)
grad = torch.autograd.grad(g(x_var), x_var)[0]
print('Gradient: ', grad)
# update with step size 0.1
x_var.data = x_var - 0.1 * grad
print('updated x: ', x_var)

x:  tensor([0., 0.])  g(x):  tensor(25.)
tensor([0., 0.], requires_grad=True)
Gradient:  tensor([-8., -6.])
updated x:  tensor([0.8000, 0.6000], requires_grad=True)
